# 3.Calidad de un vino

Dentro de las muchas variedades de vino existentes, algunas gustan más que otras, esto es debido al gusto de una persona en particular o bien a la gran cantidad de químicos y procesos que se aplican a la producción de vino. Para el área de negocios, el estimar cuál es la calidad de un vino en base a la apreciación del público es una tarea bastante difícil.
Para esta actividad se trabajará con dos datasets asociados a las variantes tinto y blanco del vino portugués ”Vinho Verde”[4]. Debido a temas privados solo se cuenta con las característcas fisioquímicas asociadas a un vino en particular, los cuales corresponden a 11 atributos numéricos descritos en el siguiente link. Este problema puede ser abordado como clasificación de 11 clases o de regresión, ya que el atributo a estimar,quality, es un valor entero entre 0 y 10.


#### (a)  Carge los dos dataset en un único dataframe de pandas, además de agregar una columna indicando si es vino tinto o blanco. Describa el dataset a trabajar.

#### (b)  Aborde este problema como si fuera de clasificación binaria para predecir si un vino es de buena calidad o no, es decir, utilice las distintas características fisioquímicas presentes en los datos para estimar esta etiqueta. Para esto cree las matrices de entrenamiento y de pruebas, además de la etiqueta para ambos conjuntos,  considerando  como quality mayor  a  5  un  vino  de  buena  calidad.  El  conjunto  de  pruebas (25 %) será utilizado únicamente para verificar la calidad de los algoritmos a entrenar.

#### (c)  Entrene un solo Árbol de Clasificación de múltiples niveles para resolver el problema. Puede variar los hiper-parámetros que prefiera, recuerde que las decisiones no pueden ser basadas mirando el conjuntode pruebas. Debido al desbalanceo que se produce en las dos clases mida la métrica F1-score [5] sobre el conjunto de entrenamiento y de pruebas.

#### (d)  Entrene un ensamblador de árboles de múltiples niveles, mediante la técnica de Random Forest. Varíe la cantidad de árboles de decisión utilizados en el ensamblado ($n_estimators$), realice un gráfico resumen del F1-score de entrenamiento y de pruebas en función de este hiper-parámetro.

#### (e)  Entrene  un  ensamblador  de  árboles  de  múltiples  niveles,  mediante  la  técnica  de $AdaBoost$ .  Varíe  la cantidad de árboles de decisión utilizados en el ensamblado ($n_estimators$), realice un gráfico resumendel F1-score de entrenamiento y de pruebas en función de este hiper-parámetro. Compare y analice con la técnica utilizada en d).

#### (f)  Entrene alguna otra máquina de aprendizaje, elegida por usted, para resolver este problema. Elija los hiper-parámetros que estime convenientes intentando aumentar el F1-score obtenido por los algoritmos anteriores. Compare y analice estas 4 maneras de resolver el problema definido en b).

#### (g)  Defina  un  criterio  para  estimar  la  importancia  de  los  distintos  atributos  en  el  ensamblado  de $Random Forest$, implementelo  sobre  alguno  de  los  ensambladores  entrenados  en  d),  haga  un  ranking  de importancia de atributos ¿Es posible implementar este criterio sobre una técnica de boost como lo es $AdaBoost$?
